In [1]:
import pandas as pd
import numpy as np
import re
import requests
import time
from bs4 import BeautifulSoup
from pandas.io.json import json_normalize
from recipe_scrapers import scrape_me
from tqdm import tqdm_notebook as tqdm
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_colwidth', -1)

In [348]:
df = pd.read_csv('/Users/stephaniekendall/Desktop/Errthang/Flatiron/projects/FP_Practice/dirty_ratings.csv')
df.drop(columns=['Unnamed: 0'], inplace=True)

In [349]:
df.sample(10)

,name,ingredients,rating
4785,Chicken and Chorizo Pasta Bake,"['3 ½ cups penne pasta', '1 tablespoon olive oil', '4 boneless chicken breasts, cut into 1/2 inch cubes', '1 red bell pepper, chopped', '1 chorizo sausage, cut into chunks', '1 ½ cups tomato puree', '1 cup light cream', '½ teaspoon red pepper flakes (optional)', '½ cup shredded Cheddar cheese', '½ cup shredded mozzarella cheese', '¼ cup freshly grated Parmesan cheese']",\n Rating: 4.31 stars\n
2867,Momma's Healthy Meatloaf,"['cooking spray', '1 tablespoon olive oil', '1 green bell pepper, diced', '1/2 cup diced sweet onion', '1/2 teaspoon minced garlic', '1 pound extra-lean (95%) ground beef', '1 cup whole wheat bread crumbs', '2 large eggs', '3/4 cup shredded carrot', '3/4 cup shredded zucchini', 'salt and ground black pepper to taste', '1/4 cup ketchup, or to taste']",4.19999980926514
4642,Shrimp and Portobello Mushroom Fettuccine,"['1 (16 ounce) package dry fettuccini noodles', '3/4 cup butter, divided', '1 pound baby portobello mushrooms, sliced', '1 large onion, chopped', '3 cloves garlic, minced', '4 ounces cream cheese', '2 tablespoons all-purpose flour', '1 pint whipping cream', '1/2 cup freshly grated Parmesan cheese, divided', '2 pounds medium shrimp, peeled and deveined', '1 tablespoon Cajun seasoning', 'salt and pepper to taste', '1/2 cup green onions, chopped']",4.25396823883057
4526,Dee's Special Chicken,"['4 skinless, boneless chicken breast halves', 'salt and pepper to taste', '1 teaspoon dried rosemary', '1 teaspoon dried sage', '1 teaspoon dried thyme', '6 cloves garlic', '1 (12 fluid ounce) can or bottle beer', '2 sprigs fresh parsley, for garnish']",3.92424249649048
6500,Best Mac 'N Cheese Ever!,"['cooking spray', '1 (16 ounce) package elbow macaroni', '1 cup evaporated milk', '4 extra large eggs', '1 (16 ounce) container sour cream', '2 teaspoons seasoned salt', '2 teaspoons ground black pepper', '2 cups shredded Cheddar cheese', '2 cups shredded Cheddar-Monterey Jack cheese blend', '6 slices processed cheese food (such as Velveeta®), torn into 4 pieces', '1/2 cup butter, melted']",4.58333349227905
5376,Grits a Ya Ya,"['3 1/2 cups chicken stock', '3/4 cup old fashioned grits', '1/4 cup heavy cream, plus more as needed', '1 cup shredded smoked Gouda cheese, or more to taste', '1/4 cup butter', '8 slices bacon, chopped', '3 tablespoons butter', '1 tablespoon minced shallot', '1 tablespoon minced garlic', '1 splash white wine', '1 pound jumbo shrimp, peeled and deveined', '2 cups chopped spinach', '1 cup chopped portobello mushrooms', '1/4 cup sliced green onions', '2 cups heavy cream', '1 dash hot pepper sauce, or to taste', 'salt and ground black pepper to taste']",4.53999996185303
792,Shrimp Fried Rice II,"['1 1/2 cups uncooked white rice', '3 cups water', '4 tablespoons vegetable oil', '1 cup fresh bean sprouts', '1/2 cup chopped onion', '1 1/2 cups cooked medium shrimp, peeled and deveined without tail', '1/4 cup chopped green onion', '2 eggs, beaten', '1 teaspoon salt', '1/4 teaspoon ground black pepper', '4 tablespoons soy sauce', '1/4 teaspoon sesame oil']",4.68400001525879
6450,Spicy Peruvian Pork,"['2 1/2 pounds boneless pork shoulder, cubed', '1/4 cup white vinegar', '1 tablespoon ground cumin', '1 tablespoon ground turmeric', '1/2 teaspoon garlic powder', '1 teaspoon salt, or to taste', '1/4 teaspoon ground black pepper, or to taste', '1 tablespoon vegetable oil', '1 cup orange juice', '1/2 cup water', '1 tablespoon dried minced onion', '1 tablespoon all-purpose flour', '2 tablespoons water']",4.52777767181396
599,Italian Style Pork Chops,"['3 cups crushed saltine crackers', '2 cups grated Parmesan cheese', '1 tablespoon Italian-style seasoning', '1/4 teaspoon garlic powder', '1 cup butter, melted', '6 pork chops']",\n Rating: 4.41 stars\n
4102,Tangy BBQ Ribs,"['8 country style pork ribs', '1 cup honey', '1 cup ketchup', '2 tablespoons molasses', '1 (18 ounce) bottle barbeque sauce']",4.37662315368652


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7426 entries, 0 to 7425
Data columns (total 3 columns):
name           7426 non-null object
ingredients    7426 non-null object
rating         7426 non-null object
dtypes: object(3)
memory usage: 174.1+ KB


In [350]:
# Remove the rows that have the rating in an unclean format (for practicing)
df = df[~df.rating.str.contains('Rating: ')]
len(df)

5000

In [351]:
df['rating'] = df.rating.astype('float')
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5000 entries, 0 to 7425
Data columns (total 3 columns):
name           5000 non-null object
ingredients    5000 non-null object
rating         5000 non-null float64
dtypes: float64(1), object(2)
memory usage: 156.2+ KB


In [21]:
df.sample(20)

,name,ingredients,rating
6126,Acorn Squash Gnocchi with Parmesan Sage Beurre Blanc,"['1 acorn squash, halved and seeded', '2 cloves garlic, pressed', '1/2 teaspoon salt', '1 egg', '1 cup whole wheat flour', '1 cup white flour', '7 cups water', '1 tablespoon salt', '1 cube chicken bouillon', '1 cup beer', '1/3 teaspoon rubbed sage', '1/4 teaspoon ground black pepper', '1/4 cup cold unsalted butter, cut into small cubes', '1/2 cup grated Parmesan cheese']",4.300000
2742,Slow Cooker German-Style Pork Roast with Sauerkraut and Potatoes,"['6 white potatoes, peeled and quartered', '1 tablespoon minced garlic', 'salt and pepper to taste', '1 (3 pound) boneless pork loin roast', '1 (32 ounce) jar sauerkraut with liquid', '2 teaspoons caraway seeds']",4.467153
3917,Unfried Chicken,"['2 cups buttermilk', '1 tablespoon hot pepper sauce', '4 skinless, boneless chicken breast halves, cut into bite-size pieces', '2 ½ cups panko bread crumbs', '6 tablespoons grated Parmesan cheese', '1 tablespoon onion powder', '2 teaspoons garlic powder', '1 teaspoon ground black pepper', '1 teaspoon cayenne pepper (optional)', '1 teaspoon paprika', 'cooking spray']",4.309524
5463,Southern Chicken and Dumplings,"['2 tablespoons butter', '1 onion, diced', '3 large bone-in chicken breast halves with skin', '1 (32 ounce) can chicken broth', '3 cups all-purpose flour', '1 tablespoon baking powder', '1 ½ teaspoons salt', '⅓ cup vegetable shortening, plus', '2 tablespoons vegetable shortening', '¾ cup milk', '1 (10.75 ounce) can condensed cream of chicken soup', '2 cups water', '½ teaspoon salt', '¼ teaspoon ground black pepper']",4.571429
4426,Pork Medallions with Port and Dried Cranberry Sauce,"['1/2 cup dried cranberries', '1 cup water', '1 teaspoon vegetable oil', '1 pound pork medallions', 'salt and pepper to taste', '2 tablespoons minced shallots', '1/2 cup tawny port wine', '1/4 cup distilled white vinegar', '1 cup chicken broth', '1/2 teaspoon dried thyme', '1 teaspoon cornstarch', '1 tablespoon water']",4.485294
5896,Creamy Beef Stroganoff,"['1 1/2 pounds ground beef', '1 teaspoon seasoned salt', '1 onion, finely chopped', '1/4 teaspoon ground nutmeg', '1/4 teaspoon ground cloves', '2 cups hot water', '1/4 cup all-purpose flour', ""2 (10.75 ounce) cans golden mushroom soup (such as Campbell's®)"", '1 (16 ounce) container sour cream', '1/4 cup dried parsley', '2 teaspoons dry mustard', '2 teaspoons salt, or to taste', '2 teaspoons ground black pepper, or to taste', '1 (16 ounce) package egg noodles']",4.558139
1889,Chicken Enchilada Casserole I,"['2 pounds skinless, boneless chicken breast halves - cooked and diced', '1 (10.75 ounce) can condensed cream of celery soup', '1 (10.75 ounce) can condensed cream of chicken soup', '1 (10 ounce) can enchilada sauce', '1 (14.5 ounce) can diced tomatoes with green chile peppers', '1 (4 ounce) can diced green chiles', '1 onion, chopped', '1 pinch garlic powder', '1 cup chicken broth', '1 teaspoon chili powder', '16 ounces processed cheese spread', '10 (6 inch) corn tortillas']",4.570755
2798,Twice-Cooked Coconut Shrimp,"['1 1/2 pounds large shrimp - peeled and deveined', '1/2 cup all-purpose flour', '1/2 cup cornstarch', '1 tablespoon salt', '1/2 tablespoon ground white pepper', '2 tablespoons vegetable oil', '1 cup ice water', '2 cups shredded coconut', '1 quart vegetable oil for frying', '1/2 cup orange marmalade', '1/4 cup Dijon-style prepared mustard', '1/4 cup honey', '1/4 teaspoon hot pepper sauce']",4.388060
1500,Crispy Fish,"['2 cups dry potato flakes', '1 cup all-purpose flour', '1 tablespoon garlic powder', '1 tablespoon seasoning salt', '1 tablespoon ground black pepper', '2 teaspoons cayenne pepper, or to taste', '4 (6 ounce) fillets cod', '2 cups butter flavored shortening, for frying']",4.437736
7337,Bolognese with Homemade Noodles,"['2 cups bread flour', '1 tablespoon flax seed meal', '2 eggs', '1 pound ground beef (85% lean)', '1 cup diced onion', '1 tablespoon minced garlic', '1 carrot, shredded', '

In [22]:
import re

# Define our patterns for a step and ingredient
# A step could have a number in front
# followed by a period like "1." or an "*"
step_pattern = re.compile(r'^(?:[0-9]{1,2}\.\s|\*)', re.S)
# An ingredient could have a fraction in front like "1/2" or  "1 1/4"
ingredient_pattern = re.compile(r'^(?:[0-9]{1,3}\s|[0-9,/]{1,4}\s|[0-9]\s[0-9][/][0-9])', re.S)

def tag(text):
    """
    Attempt to classify a line of text as a "step" or "ingredient"
    based on the formatting or leading text.
    """
    if step_pattern.match(text):
        return 'step'

    if ingredient_pattern.match(text):
        return 'ingredient'

    return None

In [27]:
tag('1/4 cup butter')

'ingredient'

In [29]:
cooking_words = ['pound', 'cup', 'chopped', 'minced', 'teaspoon',
                 'ounce', 'can', 'diced', 'without', 'inch', 'cups',
                 'shredded', 'of', 'tablespoon','drained', 'and', 'cubed', 
                 'melted', 'prepared', 'package', 'tablespoons', 'teaspoons',
                 'small', 'medium', 'large', 'juiced', 'cut', 'chunks', 'pinches',
                 'eighths', 'pounds', 'halved', 'quartered', 'seeded', 'pressed']

In [41]:
import re


SEPARATOR_RE = re.compile(r'^([\d\s*[\d\.,/]*)\s*(.+)')


def normalize(st):
    """

    :param st:
    :return:
    """
    return re.sub(r'\s+', ' ', SEPARATOR_RE.sub('\g<1> \g<2>', st)).strip()


def escape_re_string(text):
    """

    :param text:
    :return:
    """
    text = text.replace('.', '\.')
    return re.sub(r'\s+', ' ', text)

In [315]:
from itertools import chain

UNITS = {"cup": ["cups", "cup", "c.", "c"], "fluid_ounce": ["fl. oz.", "fl oz", "fluid ounce", "fluid ounces"],
         "gallon": ["gal", "gal.", "gallon", "gallons"], "ounce": ["oz", "oz.", "ounce", "ounces"],
         "pint": ["pt", "pt.", "pint", "pints"], "pound": ["lb", "lb.", "pound", "pounds"],
         "quart": ["qt", "qt.", "qts", "qts.", "quart", "quarts"],
         "tablespoon": ["tbsp.", "tbsp", "T", "T.", "tablespoon", "tablespoons", "tbs.", "tbs"],
         "teaspoon": ["tsp.", "tsp", "t", "t.", "teaspoon", "teaspoons"],
         "gram": ["g", "g.", "gr", "gr.", "gram", "grams"], "kilogram": ["kg", "kg.", "kilogram", "kilograms"],
         "liter": ["l", "l.", "liter", "liters"], "milligram": ["mg", "mg.", "milligram", "milligrams"],
         "milliliter": ["ml", "ml.", "milliliter", "milliliters"], "pinch": ["pinch", "pinches"],
         "dash": ["dash", "dashes"], "touch": ["touch", "touches"], "handful": ["handful", "handfuls"],
         "stick": ["stick", "sticks"], "clove": ["cloves", "clove"], "can": ["cans", "can"], "large": ["large"],
         "small": ["small"], "scoop": ["scoop", "scoops"], "filets": ["filet", "filets"], "sprig": ["sprigs", "sprig"]}

NUMBERS = ['seventeen', 'eighteen', 'thirteen', 'nineteen', 'fourteen', 'sixteen', 'fifteen', 'seventy', 'twelve',
           'eleven', 'eighty', 'thirty', 'ninety', 'twenty', 'seven', 'fifty', 'sixty', 'forty', 'three', 'eight',
           'four', 'zero', 'five', 'nine', 'ten', 'one', 'six', 'two', 'an', 'a', '½', '⅓','¼', '⅛', '¾']

prepositions = ["of"]

a = list(chain.from_iterable(UNITS.values()))
a.sort(key=lambda x: len(x), reverse=True)
a = map(escape_re_string, a)

PARSER_RE = re.compile(
    r'(?P<quantity>(?:[\d\.,][\d\.,\s/]*)?\s*(?:(?:%s)\s*)*)?(\s*(?P<unit>%s)\s+)?(\s*(?:%s)\s+)?(\s*(?P<name>.+))?' % (
        '|'.join(NUMBERS), '|'.join(a), '|'.join(prepositions)))


def parse(st):
    """

    :param st:
    :return:
    """
    st = normalize(st)
    res = PARSER_RE.match(st)

    return ((res.group('quantity') or '').strip() + ' ' + (res.group('unit') or '').strip(),
           (res.group('name') or '').strip())



# {
#         'measure': (res.group('quantity') or '').strip() + ' ' + (res.group('unit') or '').strip(),
#         'name': (res.group('name') or '').strip()
#     }


In [298]:
from re import search

remove = ['[ ', ']']

for val in df.ingredients:
#     val = val.strip("  ''")
    val = val.replace('[','').replace(']','').replace("'",'')
    print(parse(val))

('1 pound', 'sweet Italian sausage, 3/4 pound lean ground beef, 1/2 cup minced onion, 2 cloves garlic, crushed, 1 (28 ounce) can crushed tomatoes, 2 (6 ounce) cans tomato paste, 2 (6.5 ounce) cans canned tomato sauce, 1/2 cup water, 2 tablespoons white sugar, 1 1/2 teaspoons dried basil leaves, 1/2 teaspoon fennel seeds, 1 teaspoon Italian seasoning, 1 1/2 teaspoons salt, divided, or to taste, 1/4 teaspoon ground black pepper, 4 tablespoons chopped fresh parsley, 12 lasagna noodles, 16 ounces ricotta cheese, 1 egg, 3/4 pound mozzarella cheese, sliced, 3/4 cup grated Parmesan cheese')
('2 pounds', 'boneless pork chops, 1/2 teaspoon paprika, 1 pinch kosher salt and ground black pepper to taste, 1/4 cup butter, divided, 1 (8 ounce) package sliced fresh mushrooms, 4 cloves garlic, minced, 1 teaspoon Dijon mustard, 2 tablespoons all-purpose flour, 2 cups beef broth')
('1 1/2 pounds', 'cubed beef stew meat, salt and ground black pepper to taste, 1 onion, chopped, 1 (10.75 ounce) can condense

('1 ', 'green bell pepper, chopped, 1 yellow onion, chopped, 2 cups sliced and quartered pepperoni, 16 ounces fresh mushrooms, sliced, 1 (6 ounce) can tomato paste, 1 (32 ounce) jar spaghetti sauce, 1 clove garlic, peeled and minced, 1 cup rigatoni pasta, 1 cup rotini pasta, 1 cup macaroni, 1 pound ricotta cheese, 2 cups shredded mozzarella cheese, divided, 1/4 cup grated Parmesan cheese')
('3 ', 'eggs, 3/4 cup beer, 1 1/2 cups milk, 4 cups pastry flour, 1 tablespoon baking powder, 1/2 teaspoon baking soda, 2 tablespoons cornstarch, salt to taste, ground black pepper to taste, 1/8 teaspoon garlic powder, 1 1/2 pounds cod fillets, 2 quarts vegetable oil for frying')
('8 ', 'skinless, boneless chicken breasts, 1 (8 ounce) jar dried beef, 8 slices bacon, 8 ounces sour cream, 1 (10.75 ounce) can condensed cream of mushroom soup')
('1 pound', 'potatoes, peeled and diced, 1 teaspoon cumin, 1 teaspoon chili powder, 1 teaspoon salt, 1 tablespoon ketchup, 1 pound fresh tomatillos, husks removed

('1 pound', 'penne pasta, 3 tablespoons extra-virgin olive oil, divided, 1 pound bulk Italian sausage, 1 pound baby portobello mushrooms, sliced, 3 cloves garlic, minced, 1 1/2 cups heavy cream, 1/4 teaspoon salt, or to taste, 1/4 teaspoon cracked black pepper, 1/2 cup grated Parmesan cheese, 1 (8 ounce) package shredded whole milk mozzarella')
(' ', 'Ravioli Dough:, 2 cups all-purpose flour, 1 pinch salt, 1 teaspoon olive oil, 2 eggs, 1 1/2 tablespoons water, Ravioli Filling:, 1 (8 ounce) container ricotta cheese, 1 (4 ounce) package cream cheese, softened, 1/2 cup shredded mozzarella cheese, 1/2 cup provolone cheese, shredded, 1 egg, 1 1/2 teaspoons dried parsley, Pesto-Alfredo Cream Sauce:, 2 tablespoons olive oil, 2 cloves garlic, crushed, 3 tablespoons prepared basil pesto sauce, 2 cups heavy cream, 1/4 cup grated Parmesan cheese, 1 (24 ounce) jar marinara sauce, Egg Wash:, 1 egg, 1 tablespoon water')
('1 ', '¾ pounds skinless, boneless chicken breast halves - cubed, 1 tablespoon 

('1 cup', 'ketchup, 1 cup water, 1 tablespoon brown sugar, 1 tablespoon cider vinegar, 1/4 cup sweet pickle relish, 1 tablespoon Worcestershire sauce, 1/4 teaspoon ground cinnamon, 1 pinch ground black pepper, 1 pinch garlic salt, 1/2 teaspoon liquid smoke flavoring, 1 pound chipped ham, 6 hamburger buns')
('1/2 cup', 'plain yogurt, 1/4 cup Dijon mustard, 1 tablespoon honey, 1/4 cup fresh lemon juice, 3 tablespoons chopped fresh dill, 1 pound salmon, 1 cup white wine, 1/2 cup water, 1/4 cup chopped shallots')
('1/2 cup', 'butter, melted, 2 tablespoons lemon juice, 1/4 teaspoon garlic salt, 1/2 teaspoon dried parsley, 1/8 teaspoon paprika, 1/4 teaspoon ground white pepper, 2 pounds grouper fillets, 2 tablespoons mayonnaise, 1/8 teaspoon paprika')
('3 cups', 'penne pasta, uncooked, 1 pound Italian sausage, 1 green bell pepper, cut into thin strips, 1 red bell pepper, cut into thin strips, 1 (26 ounce) jar spaghetti sauce, 1 teaspoon fennel seed, Dash crushed red pepper, 4 ounces PHILADEL

('1/2 cup', 'olive oil, 1/4 cup lemon juice, 4 green onions, thinly sliced, 1 tablespoon chopped fresh parsley, 1 teaspoon chopped fresh rosemary, 1 teaspoon chopped fresh thyme, 1/2 teaspoon salt, 1/8 teaspoon black pepper, 1/8 teaspoon garlic powder, 3 pounds salmon fillets')
('1/2 pound', 'ground beef, 1/2 pound bulk Italian sausage, 1 onion, chopped, 4 cloves garlic, minced, 2 tablespoons chopped fresh basil, 2 tablespoons chopped fresh oregano, 2 tablespoons brown sugar, 1 tablespoon red pepper flakes, or to taste, 1 teaspoon salt, 1/2 teaspoon ground black pepper, 1 (14.5 ounce) can diced tomatoes, 1 (12 ounce) can tomato paste, 2 eggs, 2 cups ricotta cheese, 1 cup grated Parmesan cheese, 1 tablespoon chopped fresh parsley, 1 teaspoon salt, 3 large zucchini, trimmed, 2 cups shredded mozzarella cheese, divided, 2 tablespoons grated Parmesan cheese, 1 cup shredded mozzarella cheese')
(' ', '¼ cup olive oil, ¼ cup lemon juice, ¼ cup white vinegar, 2 cloves garlic, minced, 1 teaspoon

('1 cup', 'uncooked white rice, 2 tablespoons vegetable oil, 1 (10 ounce) package frozen green bell peppers and onions, 1 (15 ounce) can black beans, undrained, 1 (10 ounce) can enchilada sauce')
('4 ', 'skinless, boneless chicken breast halves - cut into cubes, 2 tablespoons butter, 2 tablespoons all-purpose flour, 1 cup chicken broth, 1 cup heavy cream, 2 tablespoons ketchup, 6 (6 inch) flour tortillas')
('5 large', 'potatoes - peeled and cubed, ½ cup milk, 1 tablespoon butter, 1 pound lean ground beef, 1 medium onion, chopped, 1 clove garlic, chopped, 1 ounce margarine, 1 ounce all-purpose flour, 2 tablespoons ketchup, 2 cups beef consomme, 2 tablespoons browning and seasoning sauce, 8 ounces shredded Cheddar cheese')
('4 ', 'baking potatoes, 1 (5 ounce) can tuna, drained, or more to taste, 1/4 cup mayonnaise, 4 green onions, chopped, or to taste, 1/2 cup shredded Cheddar cheese, or to taste')
('3 tablespoons', 'extra virgin olive oil, 2 cloves garlic, peeled and chopped, 1 (14.5 ou

(' ', '⅓ cup all-purpose flour, 2 tablespoons paprika, 1 teaspoon salt, 1 pinch ground black pepper, 6 skinless, boneless chicken breast halves, cut into bite-size pieces, 2 tablespoons vegetable oil, 1 onion, chopped, 4 cloves garlic, minced, 1 cup chicken stock, 2 tablespoons tomato paste, 1 ½ cups sour cream, 1 tablespoon paprika, 1 teaspoon cornstarch')
('1 ', '(8 ounce) package spaghetti, 1/2 cup chicken broth, 1 1/2 tablespoons white sugar, 2 tablespoons hoisin sauce, 2 tablespoons soy sauce, 2 teaspoons cornstarch, 2 tablespoons vegetable oil, 1 1/2 cups uncooked medium shrimp, peeled and deveined, 3 large cloves garlic, minced, 1/4 cup green onions, chopped')
('2 ', '(12 ounce) New York strip steaks, salt and freshly ground black pepper to taste, 1 teaspoon vegetable oil, 1 tablespoon butter, 1 cup clamshell mushrooms, separated, salt and freshly ground black pepper to taste, 1 teaspoon vegetable oil, 1 clove garlic, crushed, 2 tablespoons water, 1/2 teaspoon balsamic vinegar, 

('4 ', 'skinless, boneless chicken breast halves, salt and pepper to taste, garlic powder to taste, 3 tablespoons olive oil, 1 onion, chopped, 2 cloves garlic, minced, 1 ½ cups white wine, 1 cup chicken broth, 1 tablespoon chopped fresh rosemary leaves, 1 tablespoon chopped fresh thyme leaves, ½ cup lemon juice, 2 tablespoons grated lemon zest, ½ cup milk, ¼ cup sour cream')
('2 cups', 'molasses, 1 cup apple cider vinegar, 2 tablespoons honey, 2 tablespoons brown sugar, 1 teaspoon crushed red pepper flakes, 1 teaspoon ground ginger, 3 cloves garlic, minced, salt and pepper to taste, 1 (1 1/2 pound) pork tenderloin')
('1 cup', 'canned garbanzo beans (chickpeas), drained, rinsed, 1/8 cup tahini, 1 clove garlic, minced, 1 teaspoon salt, 1/2 teaspoon garlic powder, 1 pinch paprika, 1 1/2 teaspoons olive oil, 1 tablespoon water, 2 tablespoons fresh lemon juice, 1/3 cup chopped fresh cilantro, 1/4 cup water, 4 teaspoons red wine vinegar, 2 teaspoons extra-virgin olive oil, 2 tablespoons whit

In [310]:
for val in df.ingredients[1].split(','):
    print(parse(val.replace('[','').replace(']','').replace("'",'')))

('2 pounds', 'boneless pork chops')
('1/2 teaspoon', 'paprika')
('1 pinch', 'kosher salt and ground black pepper to taste')
('1/4 cup', 'butter')
(' ', 'divided')
('1 ', '(8 ounce) package sliced fresh mushrooms')
('4 cloves', 'garlic')
(' ', 'minced')
('1 teaspoon', 'Dijon mustard')
('2 tablespoons', 'all-purpose flour')
('2 cups', 'beef broth')


In [359]:
df.head()

,name,ingredients,rating
0,World's Best Lasagna,"['1 pound sweet Italian sausage', '3/4 pound lean ground beef', '1/2 cup minced onion', '2 cloves garlic, crushed', '1 (28 ounce) can crushed tomatoes', '2 (6 ounce) cans tomato paste', '2 (6.5 ounce) cans canned tomato sauce', '1/2 cup water', '2 tablespoons white sugar', '1 1/2 teaspoons dried basil leaves', '1/2 teaspoon fennel seeds', '1 teaspoon Italian seasoning', '1 1/2 teaspoons salt, divided, or to taste', '1/4 teaspoon ground black pepper', '4 tablespoons chopped fresh parsley', '12 lasagna noodles', '16 ounces ricotta cheese', '1 egg', '3/4 pound mozzarella cheese, sliced', '3/4 cup grated Parmesan cheese']",4.795724
1,Pork Chops in Garlic Mushroom Sauce,"['2 pounds boneless pork chops', '1/2 teaspoon paprika', '1 pinch kosher salt and ground black pepper to taste', '1/4 cup butter, divided', '1 (8 ounce) package sliced fresh mushrooms', '4 cloves garlic, minced', '1 teaspoon Dijon mustard', '2 tablespoons all-purpose flour', '2 cups beef broth']",4.572222
2,Super Duper Slow Cooker Beef Stroganoff,"['1 1/2 pounds cubed beef stew meat', 'salt and ground black pepper to taste', '1 onion, chopped', '1 (10.75 ounce) can condensed cream of mushroom soup', '1/4 cup water', '1 tablespoon dried chives', '2 cloves garlic, minced', '1 tablespoon Worcestershire sauce', '1 cube beef bouillon', '1/2 cup red wine', '1 tablespoon cornstarch', '1 tablespoon all-purpose flour', '1 (8 ounce) container sour cream', '1 (8 ounce) package sliced fresh mushrooms', '4 ounces cream cheese', '1/2 cup chopped fresh parsley']",4.596939
3,Chef John's Perfect Prime Rib,"['4 pounds prime rib roast', '1/4 cup unsalted butter, softened', '1 tablespoon freshly ground black pepper', '1 teaspoon herbes de Provence', 'kosher salt']",4.771789
4,Beef Stroganoff for Instant Pot®,"['2 tablespoons canola oil', '1/2 onion, diced', '2 teaspoons salt, divided', '2 pounds beef stew meat, cut into 1-inch cubes', '1 teaspoon freshly ground black pepper', '3 cloves garlic, minced', '1/2 teaspoon dried thyme', '2 tablespoons soy sauce', '3 cups chopped mushrooms', '2 tablespoons all-purpose flour', '3 cups chicken broth', '1 (16 ounce) package wide egg noodles', '3/4 cup sour cream, or to taste']",4.457526


In [352]:
lol = df.ingredients.values.tolist()

In [355]:
new_list = []
for sub_list in lol:
    new_list.append([])
    for item in sub_list:
            new_list[-1].append(parse(item))
lol = new_list

In [316]:
removing = ['crushed', 'divided', 'or to taste', 'sliced', 'minced', 
'softened', 'zested', 'zested and juiced', 'cored and chopped', 
'cored and cut into 1 1/2-inch pieces', 'skewered', 'juiced',
'or more as needed', 'or more if needed', 'skinless', 'foil',
'an ', 'y color - cut in half lengthwise','cored', 'an ', 'd seeded',
'<sup>&reg;</sup>eynolds Wrap(<sup>&reg;</sup>) Heavy Duty Aluminum Foil',
'quartered', 'cut into 1/2-inch cubes', 'cut into 1/4-inch thick rings',
'skin removed', 'or as needed', 'or more to taste', 'thinly sliced', '-inch thick',
'cooked', 'drained', 'beaten', '(<sup>&reg;</sup>', 'cut into thin strips', 
'softened', 'pitted and diced', 'peeled', 'halved', 'pitted', 'cut in 1/2-inch cubes',
'cored', 'plus more to taste', 'drained and rinsed', 'crushed', 'Topping:', 'Pork Chops:',
'salt and ground black pepper to taste', 'rinsed and drained', 'split', 'heated',
'cored and separated into florets', 'peeled and sliced', 'halved crosswise and split',
'Marinade:', 'stems removed', 'leaves stripped', 'zested', 'For roasting:',
'or as needed to cover', 'skinless', 'seeded and cubed', 'room temperature', 'a t',
'for brushing', 'Your favorite marinara-style pasta sauce', 'thinly sliced', 'thickly sliced',
'sliced into thin rings', 'for garnish', 'flat', 'patted dry', 'coarsely chopped', 'with skin',
'chopped', 'cut into 4 pieces', 'cut into 3 inch lengths', 'sliced into 4 inch pieces',
'thawed', 'beaten', 'julienned', 'cut into thin slices', '(optional)', '(Optional)',
'cut into bite-sized pieces', 'grated', 'boned', 'torn into small pieces', 'cut into pieces']